In [6]:
import numpy as np
import cupy as cp
import time

# Hàm cũ sử dụng NumPy
def weightss_to_lambda_numpy(weightss: np.ndarray, lambdas: np.ndarray) -> np.ndarray:
    num_qubits = weightss.shape[1]
    new_lambdas = np.zeros((4**num_qubits))
    for j, weights in enumerate(weightss):
        combinations = np.stack(np.meshgrid(*weights, indexing='ij'), axis=-1).reshape(-1, len(weights))
        new_lambdas += lambdas[j] * np.prod(combinations, axis=1)
    return new_lambdas

# Hàm mới sử dụng CuPy
def weightss_to_lambda_cupy(weightss: cp.ndarray, lambdas: cp.ndarray) -> cp.ndarray:
    num_terms, num_qubits, _ = weightss.shape
    new_lambdas = cp.zeros(4**num_qubits)
    for j in range(num_terms):
        weights = weightss[j]
        products = weights[0]
        for k in range(1, num_qubits):
            products = cp.outer(products, weights[k]).ravel()
        new_lambdas += lambdas[j] * products
    return new_lambdas

# Thiết lập tham số
num_qubits = 10  # 4^10 = 1,048,576 phần tử
num_terms = 100

# Tạo dữ liệu ngẫu nhiên cho NumPy
weightss_np = np.random.rand(num_terms, num_qubits, 4)
lambdas_np = np.random.rand(num_terms)

# Chuyển dữ liệu sang CuPy
weightss_cp = cp.asarray(weightss_np)
lambdas_cp = cp.asarray(lambdas_np)

# Benchmark hàm NumPy
start_time = time.time()
result_numpy = weightss_to_lambda_numpy(weightss_np, lambdas_np)
numpy_time = time.time() - start_time
print(f"Thời gian NumPy (CPU): {numpy_time:.4f} giây")

# Benchmark hàm CuPy
start_time = time.time()
result_cupy = weightss_to_lambda_cupy(weightss_cp, lambdas_cp)
cupy_time = time.time() - start_time
print(f"Thời gian CuPy (GPU): {cupy_time:.4f} giây")

# So sánh kết quả
result_cupy_np = cp.asnumpy(result_cupy)
if np.allclose(result_numpy, result_cupy_np):
    print("Kết quả giống nhau.")
else:
    print("Kết quả khác nhau.")

# Tính tốc độ tăng tốc
speedup = numpy_time / cupy_time
print(f"Tăng tốc: {speedup:.2f}x")

Thời gian NumPy (CPU): 9.3176 giây
Thời gian CuPy (GPU): 0.0736 giây
Kết quả giống nhau.
Tăng tốc: 126.54x
